In [10]:
import cv2
import mediapipe
import numpy as np
import math
import statistics

In [11]:
#FACE_CONNECTIONS is renamed to FACEMESH_CONTOURS
lipsUpperOuter: [61, 185, 40, 39, 37, 0, 267, 269, 270, 409, 291]
lipsLowerOuter: [146, 91, 181, 84, 17, 314, 405, 321, 375, 291]
lipsUpperInner: [78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 308]
lipsLowerInner: [78, 95, 88, 178, 87, 14, 317, 402, 318, 324, 308]
rightEyeUpper0: [246, 161, 160, 159, 158, 157, 173]
rightEyeLower0: [33, 7, 163, 144, 145, 153, 154, 155, 133]
rightEyeUpper1: [247, 30, 29, 27, 28, 56, 190]
rightEyeLower1: [130, 25, 110, 24, 23, 22, 26, 112, 243]
rightEyeUpper2: [113, 225, 224, 223, 222, 221, 189]
rightEyeLower2: [226, 31, 228, 229, 230, 231, 232, 233, 244]
rightEyeLower3: [143, 111, 117, 118, 119, 120, 121, 128, 245]
rightEyebrowUpper: [156, 70, 63, 105, 66, 107, 55, 193]
rightEyebrowLower: [35, 124, 46, 53, 52, 65]
rightEyeIris: [473, 474, 475, 476, 477]
leftEyeUpper0: [466, 388, 387, 386, 385, 384, 398]
leftEyeLower0: [263, 249, 390, 373, 374, 380, 381, 382, 362]
leftEyeUpper1: [467, 260, 259, 257, 258, 286, 414]
leftEyeLower1: [359, 255, 339, 254, 253, 252, 256, 341, 463]
leftEyeUpper2: [342, 445, 444, 443, 442, 441, 413]
leftEyeLower2: [446, 261, 448, 449, 450, 451, 452, 453, 464]
leftEyeLower3: [372, 340, 346, 347, 348, 349, 350, 357, 465]
leftEyebrowUpper: [383, 300, 293, 334, 296, 336, 285, 417]
leftEyebrowLower: [265, 353, 276, 283, 282, 295]
leftEyeIris: [468, 469, 470, 471, 472]

eye_list = [33, 246, 7, 161, 163, 160, 144, 159, 145, 158, 153, 157, 154, 173, 155, 133]

#connections=mp_face_mesh.FACEMESH_TESSELATION
#mp_face_mesh.FACEMESH_FACE_OVAL
#mp_face_mesh.FACEMESH_LEFT_EYE
#mp_face_mesh.FACEMESH_LEFT_EYEBROW
#mp_face_mesh.FACEMESH_LIPS
#mp_face_mesh.FACEMESH_RIGHT_EYE
#mp_face_mesh.FACEMESH_RIGHT_EYEBROW
#mp_face_mesh.FACEMESH_TESSELATION
#mp_face_mesh.FACEMESH_CONTOURS

In [12]:
def overlay_image_alpha(img, img_overlay, x, y, alpha_mask):
    # Image ranges
    y1, y2 = max(0, y), min(img.shape[0], y + img_overlay.shape[0])
    x1, x2 = max(0, x), min(img.shape[1], x + img_overlay.shape[1])

    # Overlay ranges
    y1o, y2o = max(0, -y), min(img_overlay.shape[0], img.shape[0] - y)
    x1o, x2o = max(0, -x), min(img_overlay.shape[1], img.shape[1] - x)

    # Exit if nothing to do
    if y1 >= y2 or x1 >= x2 or y1o >= y2o or x1o >= x2o:
        return

    # Blend overlay within the determined ranges
    img_crop = img[y1:y2, x1:x2]
    img_overlay_crop = img_overlay[y1o:y2o, x1o:x2o]
    alpha = alpha_mask[y1o:y2o, x1o:x2o, np.newaxis]
    alpha_inv = 1.0 - alpha

    img_crop[:] = alpha * img_overlay_crop + alpha_inv * img_crop

In [13]:
drawingModule = mediapipe.solutions.drawing_utils
faceModule = mediapipe.solutions.face_mesh

In [14]:
circleDrawingSpec = drawingModule.DrawingSpec(thickness=1, circle_radius=1, color=(100,75,75))
lineDrawingSpec = drawingModule.DrawingSpec(thickness=1, color=(0,255,0))

In [15]:
#From Image
#with faceModule.FaceMesh(static_image_mode=True) as face:
#    # Face landmarks estimation
#    # From image
#    image = cv2.imread("D:/Research/opencv/image/MediaPipe/sung.jpg")
    
#    results = face.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
#    if results.multi_face_landmarks != None:
#        for faceLandmarks in results.multi_face_landmarks:
#            drawingModule.draw_landmarks(image, faceLandmarks, faceModule.FACEMESH_CONTOURS, circleDrawingSpec, lineDrawingSpec)
#    cv2.imshow('Test image', image)

In [16]:
eye_img = cv2.imread("D:/Research/opencv/image/MediaPipe/Sharingan_triple.svg.png")
#alpha_mask = eye_img[:, :, 2] / 255.0

In [17]:
# From Webcam
cap = cv2.VideoCapture(1)
with faceModule.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5) as face:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            continue

        h, w, d = image.shape
        results = face.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        #if results.multi_face_landmarks != None:
        #    for faceLandmarks in results.multi_face_landmarks:
        #        drawingModule.draw_landmarks(image, faceLandmarks, faceModule.FACEMESH_CONTOURS, circleDrawingSpec, lineDrawingSpec)
        #        #drawingModule.draw_landmarks(image, faceLandmarks, faceModule.FACEMESH_LEFT_EYE | faceModule.FACEMESH_RIGHT_EYE, None, lineDrawingSpec)                
        #        #overlay_image_alpha(image, eye_img, 0, 0, alpha_mask)
        
        if results.multi_face_landmarks != None:
            # 歷遍左眼各點取得目前座標與眼睛大小
            for face_landmarks in results.multi_face_landmarks:
                #drawingModule.draw_landmarks(image, face_landmarks, faceModule.FACEMESH_LEFT_EYE | faceModule.FACEMESH_RIGHT_EYE, None, lineDrawingSpec)
                #break
                eye_point = []
                eye_size = []
                for index in eye_list:
                    x = int(face_landmarks.landmark[index].x * w)
                    y = int(face_landmarks.landmark[index].y * h)
                    eye_point.append([x, y])
                    if index == 153 or index == 159:
                        eye_size.append([x, y])
                if len(eye_size) == 2:
                    eye_len = int(math.pow(math.pow((eye_size[0][0] - eye_size[1][0]), 2) + math.pow((eye_size[0][1] - eye_size[1][1]), 2), 0.5))

            #try:
                # 將取得的各點透過statistics計算出眼睛中心座標
                points = eye_point
                center = [statistics.mean(i) for i in zip(*points)]

               # 透過剛剛取得的眼睛大小, 將眼睛圖案的圖片轉換成適合的大小
                eye = cv2.resize(eye_img, (eye_len-6, eye_len-6))

                # 透過一系列的處理將眼睛圖片貼在左眼上
                eye_gray = cv2.cvtColor(eye, cv2.COLOR_BGR2GRAY)
                _, eye_mask = cv2.threshold(eye_gray, 25, 255, cv2.THRESH_BINARY_INV)

                img_height, img_width, _ = eye.shape
                x, y = int(center[0]-img_width/2), int(center[1]-img_height/2)
                eye_area = image[y: y+img_height, x: x+img_width]
                eye_area_no_eye = cv2.bitwise_and(eye_area, eye_area, mask=eye_mask)
                final_eye = cv2.add(eye_area_no_eye, eye)
                image[y: y+img_height, x: x+img_width] = final_eye
                
                cv2.imshow('MediaPipe Face Mesh', image)
            #except:
            #    pass
        
        if cv2.waitKey(5) & 0xFF == 27:
              break     

In [18]:
cap.release()
cv2.destroyAllWindows()